In [1]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import numpy as np
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

In [3]:
def model_input(state_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_states')
    # rewards = tf.placeholder(tf.float32, [None], name='rewards')
    dones = tf.placeholder(tf.float32, [None], name='dones')
    rates = tf.placeholder(tf.float32, [None], name='rates') # success rate
    return states, actions, next_states, dones, rates

In [4]:
def Act(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Act', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        return logits

In [5]:
def Env(states, actions, state_size, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Env', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        states_logits = tf.layers.dense(inputs=nl2, units=state_size, trainable=False)
        Qlogits = tf.layers.dense(inputs=nl2, units=1, trainable=False)
        return states_logits, Qlogits

In [7]:
def model_loss(state_size, action_size, hidden_size,
               states, actions, next_states, dones, rates):
    ################################################ a = act(s)
    actions_logits = Act(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    aloss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels))
    ################################################ s', r = env(s, a)
    ################################################ ~s', Q = ~env(s, a)
    ################################################ ~s', Q = ~env(s, ~a)
    e_next_states_logits, eQs = Env(actions=actions_labels, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size)
    a_next_states_logits, aQs = Env(actions=actions_logits, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size, reuse=True)
    next_states_labels = tf.nn.sigmoid(next_states)
    eloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=e_next_states_logits, 
                                                                   labels=next_states_labels))
    aloss2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=a_next_states_logits, 
                                                                    labels=next_states_labels))
    eQs_logits = tf.reshape(eQs, shape=[-1])
    aQs_logits = tf.reshape(aQs, shape=[-1])
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=eQs_logits, # GAN
                                                                    labels=rates)) # 0-1
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs_logits, # GAN
                                                                    labels=tf.zeros_like(rates))) # min
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs_logits, # GAN
                                                                     labels=tf.ones_like(rates))) # max
    #################################################### s'', Q' = ~env(s', ~a')
    next_actions_logits = Act(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    next_states_logits, aQs2 = Env(actions=next_actions_logits, states=next_states, hidden_size=hidden_size, 
                                   action_size=action_size, state_size=state_size, reuse=True)
    aQs2_logits = tf.reshape(aQs2, shape=[-1]) * (1-dones)
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs2_logits, # GAN
                                                                    labels=tf.zeros_like(rates))) # min
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs2_logits, # GAN
                                                                     labels=tf.ones_like(rates))) # min
    #################################################### s'', Q' = ~env(s', ~a')
    next_states = tf.nn.tanh(next_states_logits)
    next_actions_logits = Act(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    next_states_logits, aQs2 = Env(actions=next_actions_logits, states=next_states, hidden_size=hidden_size, 
                                   action_size=action_size, state_size=state_size, reuse=True)
    aQs2_logits = tf.reshape(aQs2, shape=[-1]) * (1-dones)
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs2_logits, # GAN
                                                                    labels=tf.zeros_like(rates))) # min
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs2_logits, # GAN
                                                                     labels=tf.ones_like(rates))) # min
    #################################################### s'', Q' = ~env(s', ~a')
    next_states = tf.nn.tanh(next_states_logits)
    next_actions_logits = Act(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    next_states_logits, aQs2 = Env(actions=next_actions_logits, states=next_states, hidden_size=hidden_size, 
                                   action_size=action_size, state_size=state_size, reuse=True)
    aQs2_logits = tf.reshape(aQs2, shape=[-1]) * (1-dones)
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs2_logits, # GAN
                                                                    labels=tf.zeros_like(rates))) # min
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs2_logits, # GAN
                                                                     labels=tf.ones_like(rates))) # min
    #################################################### s'', Q' = ~env(s', ~a')
    next_states = tf.nn.tanh(next_states_logits)
    next_actions_logits = Act(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    next_states_logits, aQs2 = Env(actions=next_actions_logits, states=next_states, hidden_size=hidden_size, 
                                   action_size=action_size, state_size=state_size, reuse=True)
    aQs2_logits = tf.reshape(aQs2, shape=[-1]) * (1-dones)
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs2_logits, # GAN
                                                                    labels=tf.zeros_like(rates))) # min
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs2_logits, # GAN
                                                                     labels=tf.ones_like(rates))) # min
    #################################################### s'', Q' = ~env(s', ~a')
    next_states = tf.nn.tanh(next_states_logits)
    next_actions_logits = Act(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    next_states_logits, aQs2 = Env(actions=next_actions_logits, states=next_states, hidden_size=hidden_size, 
                                   action_size=action_size, state_size=state_size, reuse=True)
    aQs2_logits = tf.reshape(aQs2, shape=[-1]) * (1-dones)
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs2_logits, # GAN
                                                                    labels=tf.zeros_like(rates))) # min
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs2_logits, # GAN
                                                                     labels=tf.ones_like(rates))) # min
    #################################################### s'', Q' = ~env(s', ~a')
    next_states = tf.nn.tanh(next_states_logits)
    next_actions_logits = Act(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    next_states_logits, aQs2 = Env(actions=next_actions_logits, states=next_states, hidden_size=hidden_size, 
                                   action_size=action_size, state_size=state_size, reuse=True)
    aQs2_logits = tf.reshape(aQs2, shape=[-1]) * (1-dones)
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs2_logits, # GAN
                                                                    labels=tf.zeros_like(rates))) # min
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs2_logits, # GAN
                                                                     labels=tf.ones_like(rates))) # min
    #################################################### s'', Q' = ~env(s', ~a')
    next_states = tf.nn.tanh(next_states_logits)
    next_actions_logits = Act(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    next_states_logits, aQs2 = Env(actions=next_actions_logits, states=next_states, hidden_size=hidden_size, 
                                   action_size=action_size, state_size=state_size, reuse=True)
    aQs2_logits = tf.reshape(aQs2, shape=[-1]) * (1-dones)
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs2_logits, # GAN
                                                                    labels=tf.zeros_like(rates))) # min
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs2_logits, # GAN
                                                                     labels=tf.ones_like(rates))) # min
    return actions_logits, aloss, eloss, aloss2

In [8]:
def model_opt(a_loss, e_loss, a_loss2, a_learning_rate, e_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    a_vars = [var for var in t_vars if var.name.startswith('Act')]
    e_vars = [var for var in t_vars if var.name.startswith('Env')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        a_opt = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss, var_list=a_vars)
        e_opt = tf.train.AdamOptimizer(e_learning_rate).minimize(e_loss, var_list=e_vars)
        a_opt2 = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss2, var_list=a_vars)
    return a_opt, e_opt, a_opt2

In [9]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, a_learning_rate, e_learning_rate):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.next_states, self.dones, self.rates = model_input(state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.a_loss, self.e_loss, self.a_loss2 = model_loss(
            state_size=state_size, action_size=action_size, hidden_size=hidden_size, # model init
            states=self.states, actions=self.actions, next_states=self.next_states, 
            dones=self.dones, rates=self.rates) # model input
        
        # Update the model: backward pass and backprop
        self.a_opt, self.e_opt, self.a_opt2 = model_opt(a_loss=self.a_loss, 
                                                        e_loss=self.e_loss,
                                                        a_loss2=self.a_loss2, 
                                                        a_learning_rate=a_learning_rate,
                                                        e_learning_rate=e_learning_rate)

In [10]:
# def sample(buffer, batch_size):
#     idx = np.random.choice(np.arange(len(buffer)), size=batch_size, replace=False)
#     return [buffer[ii] for ii in idx]

In [11]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size) # data batch

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [12]:
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [13]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 4
action_size = 2
hidden_size = 4*2             # number of units in each Q-network hidden layer
a_learning_rate = 1e-4         # Q-network learning rate
e_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e3)             # experience mini-batch size: 200/500 a successfull episode size
# gamma = 0.99                   # future reward discount

In [14]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size,
              a_learning_rate=a_learning_rate, 
              e_learning_rate=e_learning_rate)

# Init the memory
memory = Memory(max_size=memory_size)

In [15]:
state = env.reset()
total_reward = 0
num_step = 0
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    rate = -1
    memory.buffer.append([state, action, next_state, reward, float(done), rate])
    num_step += 1 # memory incremented
    total_reward += reward
    state = next_state
    if done is True:
        state = env.reset()
        rate = total_reward/500
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1:
                memory.buffer[-1-idx][-1] = rate
        total_reward = 0 # reset
        num_step = 0 # reset

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list = [], []
aloss_list, eloss_list, aloss2_list = [], [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes average/running average/running mean/window

    # Training episodes/epochs
    for ep in range(11111):
        total_reward = 0 # each episode
        aloss_batch, eloss_batch, aloss2_batch = [], [], []
        state = env.reset() # each episode
        num_step = 0 # each episode
        rate = -1

        # Training steps/batches
        while True:
            # Explore (env) or Exploit (model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
                action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            memory.buffer.append([state, action, next_state, reward, float(done), rate])
            num_step += 1 # momory added
            total_reward += reward
            state = next_state
            
            # Training with the maxrated minibatch
            batch = memory.buffer
            #for idx in range(memory_size// batch_size):
            while True:
                idx = np.random.choice(np.arange(memory_size// batch_size))
                states = np.array([each[0] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                actions = np.array([each[1] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                next_states = np.array([each[2] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                #rewards = np.array([each[3] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                dones = np.array([each[4] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rates = np.array([each[5] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                states = states[rates >= np.max(rates)]
                actions = actions[rates >= np.max(rates)]
                next_states = next_states[rates >= np.max(rates)]
                #rewards = rewards[rates >= np.max(rates)]
                dones = dones[rates >= np.max(rates)]
                rates = rates[rates >= np.max(rates)]
                if np.count_nonzero(dones) > 0 and len(dones) > 1 and np.max(rates) > 0:
                    break
            aloss, _ = sess.run([model.a_loss, model.a_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               #model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            eloss, _ = sess.run([model.e_loss, model.e_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               #model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            aloss2, _= sess.run([model.a_loss2, model.a_opt2], 
                                 feed_dict = {model.states: states, 
                                              model.actions: actions,
                                              model.next_states: next_states,
                                              #model.rewards: rewards,
                                              model.dones: dones,
                                              model.rates: rates})
            # print('dones:', 
            #       len(dones), np.count_nonzero(dones), 
            #       len(dones1), np.count_nonzero(dones1), 
            #       len(dones2), np.count_nonzero(dones2))
            aloss_batch.append(aloss)
            eloss_batch.append(eloss)
            aloss2_batch.append(aloss2)
            if done is True:
                break
                
        # Rating the latest played episode
        rate = total_reward/500 # update rate at the end/ when episode is done
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1: # double-check the landmark/marked indexes
                memory.buffer[-1-idx][-1] = rate # rate the trajectory/data
                
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'rate:{:.4f}'.format(rate),
              'aloss:{:.4f}'.format(np.mean(aloss_batch)),
              'eloss:{:.4f}'.format(np.mean(eloss_batch)),
              'aloss2:{:.4f}'.format(np.mean(aloss2_batch)),
              'exploreP:{:.4f}'.format(explore_p))

        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        aloss_list.append([ep, np.mean(aloss_batch)])
        eloss_list.append([ep, np.mean(eloss_batch)])
        aloss2_list.append([ep, np.mean(aloss2_batch)])
        
        # Break episode/epoch loop
        ## Option 1: Solve the First Version
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of +30 over 100 consecutive episodes.
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:22.0000 R:22.0000 rate:0.0440 aloss:0.7046 eloss:6.8137 aloss2:6.3008 exploreP:0.9978
Episode:1 meanR:20.0000 R:18.0000 rate:0.0360 aloss:0.6961 eloss:6.8119 aloss2:6.3015 exploreP:0.9960
Episode:2 meanR:43.6667 R:91.0000 rate:0.1820 aloss:0.6938 eloss:6.7725 aloss2:6.3345 exploreP:0.9871
Episode:3 meanR:36.5000 R:15.0000 rate:0.0300 aloss:0.6915 eloss:6.7337 aloss2:6.3699 exploreP:0.9857
Episode:4 meanR:33.0000 R:19.0000 rate:0.0380 aloss:0.6905 eloss:6.7225 aloss2:6.3898 exploreP:0.9838
Episode:5 meanR:31.3333 R:23.0000 rate:0.0460 aloss:0.6931 eloss:6.7058 aloss2:6.4052 exploreP:0.9816
Episode:6 meanR:34.0000 R:50.0000 rate:0.1000 aloss:0.6973 eloss:6.6671 aloss2:6.4321 exploreP:0.9767
Episode:7 meanR:33.2500 R:28.0000 rate:0.0560 aloss:0.6897 eloss:6.6292 aloss2:6.4682 exploreP:0.9740
Episode:8 meanR:31.5556 R:18.0000 rate:0.0360 aloss:0.6887 eloss:6.6051 aloss2:6.4902 exploreP:0.9723
Episode:9 meanR:31.7000 R:33.0000 rate:0.0660 aloss:0.6878 eloss:6.5778 aloss2:6.5

Episode:80 meanR:25.6914 R:21.0000 rate:0.0420 aloss:0.7133 eloss:3.2581 aloss2:13.3785 exploreP:0.8140
Episode:81 meanR:25.8780 R:41.0000 rate:0.0820 aloss:0.7121 eloss:3.1295 aloss2:13.7144 exploreP:0.8107
Episode:82 meanR:25.8193 R:21.0000 rate:0.0420 aloss:0.7046 eloss:3.0922 aloss2:13.9281 exploreP:0.8090
Episode:83 meanR:25.7976 R:24.0000 rate:0.0480 aloss:0.6935 eloss:3.0913 aloss2:13.9924 exploreP:0.8071
Episode:84 meanR:25.6235 R:11.0000 rate:0.0220 aloss:0.6921 eloss:3.0476 aloss2:14.1773 exploreP:0.8062
Episode:85 meanR:25.5698 R:21.0000 rate:0.0420 aloss:0.6956 eloss:3.0304 aloss2:14.2412 exploreP:0.8046
Episode:86 meanR:25.4253 R:13.0000 rate:0.0260 aloss:0.6992 eloss:2.9970 aloss2:14.3923 exploreP:0.8035
Episode:87 meanR:25.4659 R:29.0000 rate:0.0580 aloss:0.6975 eloss:2.9788 aloss2:14.5038 exploreP:0.8012
Episode:88 meanR:25.4270 R:22.0000 rate:0.0440 aloss:0.6929 eloss:2.9347 aloss2:14.6858 exploreP:0.7995
Episode:89 meanR:25.2556 R:10.0000 rate:0.0200 aloss:0.6938 elos

Episode:159 meanR:32.3400 R:40.0000 rate:0.0800 aloss:0.7005 eloss:1.7411 aloss2:30.0345 exploreP:0.6186
Episode:160 meanR:32.8200 R:63.0000 rate:0.1260 aloss:0.6892 eloss:1.7296 aloss2:30.4144 exploreP:0.6148
Episode:161 meanR:32.9400 R:34.0000 rate:0.0680 aloss:0.6817 eloss:1.7265 aloss2:30.7543 exploreP:0.6128
Episode:162 meanR:32.8000 R:23.0000 rate:0.0460 aloss:0.7196 eloss:1.7769 aloss2:30.3148 exploreP:0.6114
Episode:163 meanR:32.6900 R:10.0000 rate:0.0200 aloss:0.6828 eloss:1.7445 aloss2:31.0399 exploreP:0.6108
Episode:164 meanR:32.7900 R:34.0000 rate:0.0680 aloss:0.7078 eloss:1.7776 aloss2:30.7093 exploreP:0.6087
Episode:165 meanR:32.8200 R:47.0000 rate:0.0940 aloss:0.6848 eloss:1.7251 aloss2:30.8616 exploreP:0.6059
Episode:166 meanR:32.9300 R:23.0000 rate:0.0460 aloss:0.6950 eloss:1.7276 aloss2:31.2356 exploreP:0.6046
Episode:167 meanR:32.9600 R:16.0000 rate:0.0320 aloss:0.6927 eloss:1.7121 aloss2:31.2766 exploreP:0.6036
Episode:168 meanR:33.0300 R:40.0000 rate:0.0800 aloss:0

Episode:237 meanR:47.7600 R:34.0000 rate:0.0680 aloss:0.6780 eloss:1.5964 aloss2:36.9905 exploreP:0.4128
Episode:238 meanR:48.2000 R:62.0000 rate:0.1240 aloss:0.6773 eloss:1.5728 aloss2:36.7466 exploreP:0.4103
Episode:239 meanR:48.7000 R:79.0000 rate:0.1580 aloss:0.6787 eloss:1.6148 aloss2:36.7515 exploreP:0.4072
Episode:240 meanR:48.4400 R:23.0000 rate:0.0460 aloss:0.6758 eloss:1.5885 aloss2:36.5036 exploreP:0.4063
Episode:241 meanR:48.4900 R:37.0000 rate:0.0740 aloss:0.6716 eloss:1.6010 aloss2:36.7393 exploreP:0.4048
Episode:242 meanR:49.2600 R:103.0000 rate:0.2060 aloss:0.6783 eloss:1.6058 aloss2:36.6212 exploreP:0.4008
Episode:243 meanR:49.9400 R:81.0000 rate:0.1620 aloss:0.6737 eloss:1.6421 aloss2:36.3226 exploreP:0.3976
Episode:244 meanR:50.7700 R:106.0000 rate:0.2120 aloss:0.6781 eloss:1.6049 aloss2:36.5588 exploreP:0.3935
Episode:245 meanR:51.3300 R:94.0000 rate:0.1880 aloss:0.6770 eloss:1.6089 aloss2:36.6881 exploreP:0.3899
Episode:246 meanR:52.0100 R:83.0000 rate:0.1660 aloss

Episode:315 meanR:94.0500 R:143.0000 rate:0.2860 aloss:0.6940 eloss:1.3944 aloss2:63.9095 exploreP:0.1924
Episode:316 meanR:93.9700 R:97.0000 rate:0.1940 aloss:0.6951 eloss:1.4006 aloss2:64.9398 exploreP:0.1906
Episode:317 meanR:94.7300 R:198.0000 rate:0.3960 aloss:0.7017 eloss:1.3876 aloss2:66.5284 exploreP:0.1871
Episode:318 meanR:96.0800 R:173.0000 rate:0.3460 aloss:0.7001 eloss:1.3738 aloss2:68.1154 exploreP:0.1841
Episode:319 meanR:97.0200 R:150.0000 rate:0.3000 aloss:0.7034 eloss:1.3944 aloss2:69.0709 exploreP:0.1815
Episode:320 meanR:97.8200 R:109.0000 rate:0.2180 aloss:0.7039 eloss:1.3918 aloss2:70.3008 exploreP:0.1796
Episode:321 meanR:98.9200 R:214.0000 rate:0.4280 aloss:0.7063 eloss:1.3859 aloss2:71.8981 exploreP:0.1760
Episode:322 meanR:99.7700 R:108.0000 rate:0.2160 aloss:0.6985 eloss:1.3744 aloss2:72.7152 exploreP:0.1742
Episode:323 meanR:99.9600 R:78.0000 rate:0.1560 aloss:0.7108 eloss:1.3467 aloss2:73.7063 exploreP:0.1730
Episode:324 meanR:100.6100 R:166.0000 rate:0.332

...

Episode:1080 meanR:483.9000 R:500.0000 rate:1.0000 aloss:0.0695 eloss:0.8126 aloss2:633.4254 exploreP:0.0100
Episode:1081 meanR:483.9000 R:500.0000 rate:1.0000 aloss:0.0697 eloss:0.8137 aloss2:673.5231 exploreP:0.0100
Episode:1082 meanR:483.9000 R:500.0000 rate:1.0000 aloss:0.0697 eloss:0.8137 aloss2:513.7798 exploreP:0.0100
Episode:1083 meanR:480.1000 R:120.0000 rate:0.2400 aloss:0.0680 eloss:0.8171 aloss2:508.5876 exploreP:0.0100
Episode:1084 meanR:479.4900 R:439.0000 rate:0.8780 aloss:0.0683 eloss:0.8146 aloss2:511.9467 exploreP:0.0100
Episode:1085 meanR:481.3300 R:500.0000 rate:1.0000 aloss:0.0711 eloss:0.8167 aloss2:1054.1680 exploreP:0.0100
Episode:1086 meanR:481.3300 R:500.0000 rate:1.0000 aloss:0.0697 eloss:0.8158 aloss2:527.9359 exploreP:0.0100
Episode:1087 meanR:481.3300 R:500.0000 rate:1.0000 aloss:0.0690 eloss:0.8168 aloss2:719.2759 exploreP:0.0100
Episode:1088 meanR:481.3300 R:500.0000 rate:1.0000 aloss:0.0699 eloss:0.8088 aloss2:1710.5780 exploreP:0.0100
Episode:1089 mean

Episode:1155 meanR:485.8800 R:500.0000 rate:1.0000 aloss:0.0745 eloss:0.7205 aloss2:2695.7634 exploreP:0.0100
Episode:1156 meanR:485.8800 R:500.0000 rate:1.0000 aloss:0.0756 eloss:0.7178 aloss2:2891.2300 exploreP:0.0100
Episode:1157 meanR:485.8800 R:500.0000 rate:1.0000 aloss:0.0715 eloss:0.7155 aloss2:3062.7000 exploreP:0.0100
Episode:1158 meanR:485.8800 R:500.0000 rate:1.0000 aloss:0.0752 eloss:0.7178 aloss2:3286.5266 exploreP:0.0100
Episode:1159 meanR:485.8800 R:500.0000 rate:1.0000 aloss:0.0752 eloss:0.7188 aloss2:3483.3301 exploreP:0.0100
Episode:1160 meanR:485.8800 R:500.0000 rate:1.0000 aloss:0.0744 eloss:0.7111 aloss2:3463.6956 exploreP:0.0100
Episode:1161 meanR:485.8800 R:500.0000 rate:1.0000 aloss:0.0747 eloss:0.7143 aloss2:3452.5286 exploreP:0.0100
Episode:1162 meanR:485.8800 R:500.0000 rate:1.0000 aloss:0.0752 eloss:0.7128 aloss2:3325.6433 exploreP:0.0100
Episode:1163 meanR:485.8800 R:500.0000 rate:1.0000 aloss:0.0747 eloss:0.7130 aloss2:3123.2705 exploreP:0.0100
Episode:11

Episode:1230 meanR:479.8000 R:500.0000 rate:1.0000 aloss:0.1022 eloss:0.7050 aloss2:1547.9641 exploreP:0.0100
Episode:1231 meanR:479.8000 R:500.0000 rate:1.0000 aloss:0.0981 eloss:0.7032 aloss2:1565.7083 exploreP:0.0100
Episode:1232 meanR:479.8000 R:500.0000 rate:1.0000 aloss:0.1011 eloss:0.7054 aloss2:1539.1700 exploreP:0.0100
Episode:1233 meanR:475.9700 R:117.0000 rate:0.2340 aloss:0.1000 eloss:0.7028 aloss2:1531.3406 exploreP:0.0100
Episode:1234 meanR:475.9700 R:500.0000 rate:1.0000 aloss:0.1025 eloss:0.7047 aloss2:1530.5051 exploreP:0.0100
Episode:1235 meanR:475.9700 R:500.0000 rate:1.0000 aloss:0.1060 eloss:0.7050 aloss2:1523.6755 exploreP:0.0100
Episode:1236 meanR:475.9700 R:500.0000 rate:1.0000 aloss:0.1047 eloss:0.7029 aloss2:1522.4143 exploreP:0.0100
Episode:1237 meanR:471.4600 R:49.0000 rate:0.0980 aloss:0.1139 eloss:0.7042 aloss2:1639.8881 exploreP:0.0100
Episode:1238 meanR:471.4600 R:500.0000 rate:1.0000 aloss:0.1146 eloss:0.7033 aloss2:1474.9266 exploreP:0.0100
Episode:123

Episode:1305 meanR:441.0000 R:63.0000 rate:0.1260 aloss:0.0925 eloss:0.8993 aloss2:845.9372 exploreP:0.0100
Episode:1306 meanR:441.0000 R:500.0000 rate:1.0000 aloss:0.0920 eloss:0.8885 aloss2:839.9081 exploreP:0.0100
Episode:1307 meanR:441.0000 R:500.0000 rate:1.0000 aloss:0.0905 eloss:0.8855 aloss2:830.8370 exploreP:0.0100
Episode:1308 meanR:441.0000 R:500.0000 rate:1.0000 aloss:0.0882 eloss:0.8833 aloss2:820.1772 exploreP:0.0100
Episode:1309 meanR:441.0000 R:500.0000 rate:1.0000 aloss:0.0866 eloss:0.8923 aloss2:814.9418 exploreP:0.0100
Episode:1310 meanR:441.0000 R:500.0000 rate:1.0000 aloss:0.0846 eloss:0.8965 aloss2:814.6625 exploreP:0.0100
Episode:1311 meanR:441.0000 R:500.0000 rate:1.0000 aloss:0.0872 eloss:0.8992 aloss2:817.0483 exploreP:0.0100
Episode:1312 meanR:440.0500 R:405.0000 rate:0.8100 aloss:0.0897 eloss:0.9097 aloss2:816.7524 exploreP:0.0100
Episode:1313 meanR:440.0500 R:500.0000 rate:1.0000 aloss:0.0836 eloss:0.8972 aloss2:810.5540 exploreP:0.0100
Episode:1314 meanR:4

Episode:1381 meanR:485.7600 R:500.0000 rate:1.0000 aloss:0.0738 eloss:0.9029 aloss2:650.3369 exploreP:0.0100
Episode:1382 meanR:485.7600 R:500.0000 rate:1.0000 aloss:0.0759 eloss:0.9028 aloss2:680.2568 exploreP:0.0100
Episode:1383 meanR:485.7600 R:500.0000 rate:1.0000 aloss:0.0730 eloss:0.9014 aloss2:664.1347 exploreP:0.0100
Episode:1384 meanR:485.7600 R:500.0000 rate:1.0000 aloss:0.0746 eloss:0.9019 aloss2:670.6844 exploreP:0.0100
Episode:1385 meanR:490.2400 R:500.0000 rate:1.0000 aloss:0.0751 eloss:0.9090 aloss2:707.5938 exploreP:0.0100
Episode:1386 meanR:490.2400 R:500.0000 rate:1.0000 aloss:0.0723 eloss:0.8974 aloss2:676.3544 exploreP:0.0100
Episode:1387 meanR:486.5600 R:132.0000 rate:0.2640 aloss:0.0728 eloss:0.8949 aloss2:652.1764 exploreP:0.0100
Episode:1388 meanR:486.5600 R:500.0000 rate:1.0000 aloss:0.0747 eloss:0.8928 aloss2:676.7607 exploreP:0.0100
Episode:1389 meanR:486.5600 R:500.0000 rate:1.0000 aloss:0.0741 eloss:0.8923 aloss2:687.9617 exploreP:0.0100
Episode:1390 meanR:

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(aloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses')

In [ ]:
eps, arr = np.array(eloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Env losses')

In [ ]:
eps, arr = np.array(aloss2_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses 2')

## Testing

Let's checkout how our trained agent plays the game.

In [39]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(10):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
